Technical Guide: EnMAP Atmospheric Correction using 6ABOSThis guide explains how to configure and run the 6ABOS (6S-Atmospheric Band Optimized System) pipeline for EnMAP L1C data.1. Configuration Dictionary (conf)The conf dictionary is the central control for the processing pipeline. You must define your local paths and processing preferences here:VariableDescriptioninput_dirMandatory. The local path to the folder containing your EnMAP L1C data (must include the .TIF radiance image and the METADATA.XML).output_dirMandatory. The directory where 6ABOS will save the resulting corrected GeoTIFF.VNIRBoolean. If True, the script only processes the VNIR detector bands. If False, it processes the full spectrum (VNIR + SWIR).GEEBoolean. If True, it uses Google Earth Engine to fetch real-time atmospheric data (AOT, Water Vapor, Ozone). Requires a GEE account.tgas_thresholdFloat. Gas transmittance threshold (0-1). Bands with a transmittance lower than this value (usually due to water vapor absorption) will be flagged or filtered.data storingBoolean. If True, the final result will be exported as a 32-bit Float GeoTIFF datacube.verboseBoolean. If True, the console will print detailed progress for every processed band.Python# Configuration Example
conf = {
    "input_dir": r'C:\Users\PhD_Gabriel\Data\EnMAP_Scene_01',
    "output_dir": r'C:\Users\PhD_Gabriel\Data\Results',
    "VNIR": False,
    "GEE": True,
    "tgas_threshold": 0.75,
    "data storing": True,
    "verbose": True
}
2. Physical Basis: Surface Reflectance FormulaThe core of the correction involves inverting the Top-of-Atmosphere (TOA) signal to obtain the Bottom-of-Atmosphere (BOA) reflectance ($\rho_{BOA}$), also known as $R_{rs}$.The following physical equation is applied to each pixel:$$\rho_{BOA} = \frac{\frac{L_{TOA}}{T_{g,O3}} - L_{path}}{\frac{E_s \cdot T_{up}}{\pi} + s_{atm} \cdot (\frac{L_{TOA}}{T_{g,O3}} - L_{path})}$$Where:$L_{TOA}$: At-sensor radiance (corrected by Earth-Sun distance).$T_{g,O3}$: Gaseous transmittance (mainly Ozone).$L_{path}$: Path radiance (atmospheric intrinsic radiance).$E_s$: Corrected solar irradiance.$T_{up}$: Upward atmospheric transmittance.$s_{atm}$: Spherical albedo of the atmosphere.3. Workflow StepsMetadata Extraction: The script reads the METADATA.XML to obtain geometry (angles) and spectral calibration (CWL, FWHM).Atmospheric Setup: Parameters are fetched from GEE or metadata to build the 6S atmospheric profile.Coefficient Calculation: Py6S computes the correction coefficients for each specific EnMAP band.Correction & Export: The formula is applied to the image cube, and a new GeoTIFF is saved in the output_dir.